In [1]:
import pandas as pd
import torch

with open("dataset.txt", "r", encoding="utf-8") as f:
    text = f.read()

print("Length of dataset: ", len(text))

print(text[:1000])

Length of dataset:  83454369
thơ lục bát: 
 ai ơi xa bến quê hương
nhớ về quê mẹ nắng sương đây nè
nhớ sao những buổi trưa hè
võng đưa cót két gió hè hiu hiu
lời ru của mẹ dấu yêu
ngọt ngào êm dịu mẹ yêu con nhiều
lời mẹ năm tháng sớm chiều
con nghe nhớ mãi những điều ru ta
lòng mẹ ôi thật bao la
thái bình rộng lớn thương là mênh mông
một đời lưng mẹ đã còng
vai mang tay xách lo chồng chăm con
mong sao con lớn nên người
thân mẹ có cực vẫn cười thương con
vì đời mẹ sống cho con
gom bao mệt nhọc mãi còn vòng tay
mong chờ cho đến một ngày
công thành danh toại là ngày mẹ mong
dù nay mẹ đã xa rồi
con đây nhớ mãi mẹ ôi vạn lần
nguyện rằng ghi nhớ công ân
sinh thành dưỡng dục mẫu thân đời đời
thơ lục bát: 
 mùa đông để mộng nằm im
bao nhiêu nỗi nhớ biết tìm nơi đâu
trăng bơ vơ lạnh mái đầu
tuyết rơi bông nhớ gió sầu khúc ru

một mình lẻ bóng phiêu du
một mình thổn thức tâm tư quê người
tiếc giọt nắng tuổi đôi mươi
rơi trên thềm mộng cuộc đời trắng trong

nhớ ngọn gió bế tuổi hồng
hương hoa th

In [2]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("nhớ về quê hương"))
print(decode(encode("nhớ về quê hương")))


data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])


 0123456789:=`abcdefghijklmnopqrstuvwxyz|~¡¢¤¥¬­°²³´µ¸¹àáâãäåçèéêëìíîïñòóôõöøùúûüýÿāăąćđēęěĩīńōœšťũūůơƣưǎǒǚǵṇ̀̀́̂̃̉γсцḥḷḿṃṇṕṭạảấầẩẫậắằẳẵặẹẻẽếềểễệỉịọỏốồổỗộớờởỡợụủứừửữựỳỵỷỹ​‌‍‎₫◇❄❣❤。️﻿，￼🍁🍂💋😉😔😥😧🤣
194
[29, 23, 156, 2, 37, 143, 2, 32, 36, 66, 2, 23, 105, 103, 29, 22]
nhớ về quê hương
torch.Size([83454369]) torch.int64
tensor([ 35,  23, 103,   2,  27, 161,  18,   2,  17,  58,  35,  13,   2,   1,
          2,  16,  24,   2, 103,  24,   2,  39,  16,   2,  17, 142,  29,   2,
         32,  36,  66,   2,  23, 105, 103,  29,  22,   1,  29,  23, 156,   2,
         37, 143,   2,  32,  36,  66,   2,  28, 139,   2,  29, 134,  29,  22,
          2,  34, 105, 103,  29,  22,   2,  89,  59,  40,   2,  29,  64,   1,
         29,  23, 156,   2,  34,  16,  30,   2,  29,  23, 166,  29,  22,   2,
         17,  36, 153,  24,   2,  35,  33, 105,  16,   2,  23,  64,   1,  37,
         76,  29])


In [3]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

block_size = 8
print(train_data[:block_size+1])

x = train_data[:block_size+1]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t + 1]
    target = y[t]
    print(f"when input is {context} the target is {target}")

tensor([ 35,  23, 103,   2,  27, 161,  18,   2,  17])
when input is tensor([35]) the target is 23
when input is tensor([35, 23]) the target is 103
when input is tensor([ 35,  23, 103]) the target is 2
when input is tensor([ 35,  23, 103,   2]) the target is 27
when input is tensor([ 35,  23, 103,   2,  27]) the target is 161
when input is tensor([ 35,  23, 103,   2,  27, 161]) the target is 18
when input is tensor([ 35,  23, 103,   2,  27, 161,  18]) the target is 2
when input is tensor([ 35,  23, 103,   2,  27, 161,  18,   2]) the target is 17


In [4]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):

    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch("train")
print("inputs:")
print(xb.shape)
print(xb)
print("targets:")
print(yb.shape)

print('--------------------------------')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context} the target is {target}")

inputs:
torch.Size([4, 8])
tensor([[  2,  18,  23, 148,  36,   2,  89,  74],
        [ 74,  29,  22,   2,  29,  22, 130,  28],
        [ 28,   2,  18, 105, 157,  24,   2,  27],
        [157,  24,   2,  35,  74,  18,   2,  39]])
targets:
torch.Size([4, 8])
--------------------------------
when input is tensor([2]) the target is 18
when input is tensor([ 2, 18]) the target is 23
when input is tensor([ 2, 18, 23]) the target is 148
when input is tensor([  2,  18,  23, 148]) the target is 36
when input is tensor([  2,  18,  23, 148,  36]) the target is 2
when input is tensor([  2,  18,  23, 148,  36,   2]) the target is 89
when input is tensor([  2,  18,  23, 148,  36,   2,  89]) the target is 74
when input is tensor([  2,  18,  23, 148,  36,   2,  89,  74]) the target is 24
when input is tensor([74]) the target is 29
when input is tensor([74, 29]) the target is 22
when input is tensor([74, 29, 22]) the target is 2
when input is tensor([74, 29, 22,  2]) the target is 29
when input is tenso

In [12]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()

        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets = None):

        logits = self.token_embedding_table(idx)
        
        if targets is None:
            loss = None
            return logits, loss

        B, T, C = logits.shape
        logits = logits.view(B*T, C)
        targets = targets.view(B*T)
        loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx
    


In [13]:
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)
idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(idx, 100)[0].tolist()))

torch.Size([32, 194])
tensor(5.9145, grad_fn=<NllLossBackward0>)
̣skữćêỷằørãećợ2ả￼ťēàöů😔`γrcỷ`îừxẽẻ~¹ǵđ😉m77zцệ‌êỹ¬ừwц²ẽñạăụ😉ạ̈ợ´²5ṃẹëñǒ🍂ǒ‍snçậ¢p7ặǎ¢😉ứõ̀ừ=ýäạḿàứàễé


In [14]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [15]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
batch_size = 32

for steps in range(100):
    xb, yb = get_batch()
    
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
    print(loss.item())